In [2]:
!pip3 install --user underthesea

In [23]:
import regex as re
from underthesea import word_tokenize
import pandas as pd

In [24]:
# read txt file
f = open("input/product_train_0.txt", "r")

# get category and product
categories = []
products = []

while True:
    line = f.readline()
    if not line:
        break
    category = line.split(" ", 1)[0]
    categories.append(category)

    product = ''.join(line.split(" ", 1)[1:]).replace("\n", "")
    products.append(product)

# remove duplicate
unique_categories = list(dict.fromkeys(categories))

print(len(categories))
print(len(products))

print(categories[0:5])
print(products[0:5])

14976
14976
['__label__Ghế_văn_phòng', '__label__Thước_dây', '__label__Cưa_tay', '__label__Váy,_đầm', '__label__Dụng_cụ_đo,_kiểm_tra_khác']
['Ghế HQ - HK095', '30m Thước dây làm bằng sợi thủy tinh TOTAL TMTF12306', 'Cưa gỗ cầm tay cán lớn Asaki AK - 8657', 'Đầm ren cổ V tay ngắn cao cấp', 'Máy đo độ đồng tâm HANN YAN 6401D1']


In [25]:
data = pd.DataFrame(list(zip(products, categories)),
                    columns=['products', 'categories'])
print(data.head(5))

                                            products  \
0                                     Ghế HQ - HK095   
1  30m Thước dây làm bằng sợi thủy tinh TOTAL TMT...   
2             Cưa gỗ cầm tay cán lớn Asaki AK - 8657   
3                      Đầm ren cổ V tay ngắn cao cấp   
4                 Máy đo độ đồng tâm HANN YAN 6401D1   

                           categories  
0              __label__Ghế_văn_phòng  
1                  __label__Thước_dây  
2                    __label__Cưa_tay  
3                   __label__Váy,_đầm  
4  __label__Dụng_cụ_đo,_kiểm_tra_khác  


# EDA


In [26]:
# check misssing values

data.isna().sum()

products      0
categories    0
dtype: int64

In [27]:
# check data shape

data.shape

(14976, 2)

In [28]:
# check target balance

print(data['categories'].value_counts())

categories
__label__Bao_đựng,_ốp_lưng_điện_thoại    342
__label__Bàn_ghế_phòng_khách             187
__label__Bộ_bàn_ghế_cafe                 175
__label__Váy,_đầm                        158
__label__Bộ_bàn_ghế_(_phòng_ăn,_bếp_)    135
                                        ... 
__label__Tời_kéo                           1
__label__Van_công_nghiệp_khác              1
__label__Máy_in_khổ_lớn                    1
__label__Bể_cá                             1
__label__Bánh_pía                          1
Name: count, Length: 1971, dtype: int64


# Data Preprocessing


## 1. Chuẩn hoá unicode


In [29]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)


def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)


bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

## 2. Chuẩn hoá kiểu gõ dấu

VD: **òa** và **oà** là 2 từ khác nhau. <br>
Chi tiết: [Wikipedia](https://vi.wikipedia.org/wiki/Quy_t%E1%BA%AFc_%C4%91%E1%BA%B7t_d%E1%BA%A5u_thanh_c%E1%BB%A7a_ch%E1%BB%AF_Qu%E1%BB%91c_ng%E1%BB%AF)


In [30]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^p{P}*)([p{L}.]*p{L}+)(p{P}*$)',
                    r'1/2/3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


"""
    End section: Chuyển câu văn về cách gõ dấu kiểu cũ: dùng òa úy thay oà uý
    Xem tại đây: https://vi.wikipedia.org/wiki/Quy_tắc_đặt_dấu_thanh_trong_chữ_quốc_ngữ
"""
# if __name__ == '__main__':
print(chuan_hoa_dau_cau_tieng_viet('anh hòa thụy, đang làm.. gì'))

anh hòa thụy, đang làm.. gì


## 3. Tách từ

Học sinh học sinh học => Học_sinh học sinh_học


In [31]:
from underthesea import word_tokenize

sentence = "Tôi là sinh viên trường đại học bách khoa hà nội"
print(word_tokenize(sentence, format="text"))

Tôi là sinh_viên trường đại_học bách_khoa hà_nội


## 4. Đưa về chữ thường


In [32]:
sentence=sentence.lower()
print(sentence)

tôi là sinh viên trường đại học bách khoa hà nội


# Hàm tiền xử lý


In [33]:
import regex as re


def text_preprocess(document):
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    # document = re.sub(
    #     r'[^áàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]', ' ', document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

documents = """
TP HCM phạt người không đeo khẩu trang nơi công cộng
Người dân ở thành phố không đeo khẩu trang nơi công cộng sẽ bị xử phạt mức cao nhất 300.000 đồng, từ ngày 5/8.

Yêu cầu này được Chủ tịch UBND thành phố Nguyễn Thành Phong đưa ra tại cuộc họp Ban chỉ đạo phòng chống dịch bệnh Covid-19 của TP HCM chiều 3/8.

Việc xử phạt không đeo khẩu trang nơi công cộng được TP HCM cũng như các địa phương khác thực hiện từ cuối tháng 3 khi Covid-19 bùng phát. Tuy nhiên, sau khi hết thực hiện cách ly xã hội từ ngày 23/4, việc đeo khẩu trang nơi công cộng chỉ dừng lại ở mức khuyến cáo.

Theo Nghị định số 176/2013, người dân không đeo khẩu trang nơi công cộng sẽ bị xử phạt từ 100.000 đến 300.000 đồng. Trong khoảng một tháng áp dụng trước đó, TP HCM đã xử phạt hơn 4.300 trường hợp với gần 870 triệu đồng.

Theo ông Phong, việc đeo khẩu trang đã được khẳng định có thể tránh lây lan dịch bệnh cho người khác và bảo vệ sức khỏe cho người sử dụng. "Sở Công thương phải nắm nguồn cung ứng khẩu trang, chủ động thông báo các điểm bán để người dân dễ dàng mua vì đã xử phạt thì phải bảo đảm đủ nguồn cung", ông Phong nói.

Đội trật tự đô thị phường Bến Nghé, quận 1, xử phạt người không đeo khẩu trang trên phố đi bộ Nguyễn Huệ, chiều 15/4. Ảnh: Quỳnh Trần.
Đội trật tự đô thị phường Bến Nghé, quận 1, xử phạt người không đeo khẩu trang trên phố đi bộ Nguyễn Huệ, chiều 15/4. Ảnh: Quỳnh Trần.

Bí thư Thành uỷ Nguyễn Thiện Nhân cũng cho rằng việc đeo khẩu trang là một trong những biện pháp cơ bản để tránh dịch bệnh lây lan. Việc này rất dễ làm, không tốn nhiều tiền nhưng nhiều nước bỏ lơi và đã bị "vỡ trận".

"Ngoài đường hiện có ít nhất 20% người không đeo khẩu trang. Người không đeo không những tự rước bệnh vào mình mà còn nguy cơ lây cho người khác. Đeo khẩu trang hơi cực tí thôi nhưng đi đâu cũng nên đeo để giữ an toàn", ông Nhân nói và khẳng định thành phố bảo đảm không thiếu khẩu trang cho người dân.

Chủ tịch UBND thành phố Nguyễn Thành Phong cũng cho biết đã đồng ý việc tái lập các chốt kiểm soát ở cửa ngõ TP HCM để phòng chống Covid-19.

Trước đó, thành phố đã lập 62 chốt kiểm soát, hoạt động 24/24 từ ngày 4/4 để phòng chống dịch. Lực lượng tham gia là Công an thành phố, Sở Y tế, Bộ Tư lệnh thành phố, Thanh tra giao thông, Ban Quản lý An toàn thực phẩm, quản lý thị trường.

Trong đó, 16 chốt chính (cấp thành phố) đặt tại: Trạm thu phí Long Phước (cao tốc TP HCM - Long Thành - Dầu Giây), cao tốc Trung Lương, cầu Đôi (đường Trần Văn Giàu), đường Ba Làng, đường Xuyên Á (quốc lộ 22), cầu Phú Cường, cầu Vĩnh Bình, cầu vượt Sóng Thần, quốc lộ 1K, quốc lộ 50, quốc lộ 1A, cầu Đồng Nai, Bến xe Miền Tây, Bến xe miền Đông, sân bay Tân Sơn Nhất, cảng Cát Lái.

Đến ngày 23/4, chính quyền thành phố dừng hoạt động các chốt này vì dịch bệnh đã được khống chế, TP HCM dừng cách ly xã hội theo Chỉ thị 19 của Thủ tướng.

Sau 19 ngày hoạt động, các chốt chính đã kiểm tra gần 270.000 xe, trong đó có 235.000 ôtô; gần 600.000 người được kiểm tra y tế, đo thân nhiệt, bao gồm cả 3.000 người nước ngoài; hơn 130.000 người được yêu cầu khai báo y tế.
"""
print(text_preprocess(documents))

tp hcm phạt người không đeo khẩu_trang nơi công_cộng người_dân ở thành_phố không đeo khẩu_trang nơi công_cộng sẽ bị xử_phạt mức cao nhất 300.000 đồng , từ ngày 58 . yêu_cầu này được chủ_tịch ubnd thành_phố nguyễn_thành_phong đưa ra tại cuộc họp ban chỉ_đạo phòng_chống dịch_bệnh covid-19 của tp hcm chiều 38 . việc xử_phạt không đeo khẩu_trang nơi công_cộng được tp hcm cũng như các địa_phương khác thực_hiện từ cuối tháng 3 khi covid-19 bùng_phát . tuy_nhiên , sau khi hết thực_hiện cách_ly xã_hội từ ngày 234 , việc đeo khẩu_trang nơi công_cộng chỉ dừng lại ở mức khuyến_cáo . theo nghị_định số 1762013 , người_dân không đeo khẩu_trang nơi công_cộng sẽ bị xử_phạt từ 100.000 đến 300.000 đồng . trong khoảng một tháng áp_dụng trước đó , tp hcm đã xử_phạt hơn 4.300 trường_hợp với gần 870 triệu đồng . theo ông phong , việc đeo khẩu_trang đã được khẳng_định có_thể tránh lây_lan dịch_bệnh cho người khác và bảo_vệ sức_khỏe cho người sử_dụng . " sở công_thương phải nắm nguồn cung_ứng khẩu_trang , chủ

In [34]:
# process all products
products = [text_preprocess(product) for product in products]
print(products[0:5])

['ghế hq - hk095', '30 m thước_dây làm bằng sợi thủy tinh_total tmtf12306', 'cưa gỗ cầm tay cán lớn asaki ak - 8657', 'đầm ren cổ v tay ngắn cao_cấp', 'máy đo độ đồng_tâm hann yan 6401 d1']


## loại bỏ stopwords

# Xây dựng mô hình phân loại


## 1. Xây dựng tập train / test


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

test_percent = 0.2

# split data
X_train, X_test, y_train, y_test = train_test_split(products, categories, test_size=test_percent,
                                                    random_state=42)

# save train/test data
with open('data/train.txt', 'w') as f:
    for x, y in zip(X_train, y_train):
        y = y.replace(" ", "_")
        f.write('__label__' + y + ' ' + x + '\n')

with open('data/test.txt', 'w') as f:
    for x, y in zip(X_test, y_test):
        y = y.replace(" ", "_")
        f.write('__label__' + y + ' ' + x + '\n')

# encode target
label_encoder = LabelEncoder()
label_encoder.fit(data['categories'])

print(list(label_encoder.classes_), 'n')

y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

print(X_train[0:5])

['__label__Access_point_(Wifi)', '__label__Ampe_kìm', '__label__Amplifier', '__label__Amplifier_Hi-End', '__label__An_toàn_cho_bé_khi_đi_ô_tô', '__label__An_toàn_cho_trẻ', '__label__Ba_lô_học_sinh', '__label__Ba_lô_leo_núi,_du_lịch', '__label__Baga_xe_máy', '__label__Balo_thời_trang', '__label__Bao_bì_thực_phẩm', '__label__Bao_cao_su', '__label__Bao_cố_định_chấn_thương', '__label__Bao_đựng,_ốp_lưng_điện_thoại', '__label__Barie_tự_động', '__label__Bia', '__label__Biến_tần', '__label__Biến_áp_máy_cấp_nguồn_đa_năng', '__label__Biển,_bảng,_đèn_thông_báo_(_Led_)', '__label__Biển_báo_giao_thông', '__label__Biển_chỉ', '__label__Block_điều_hòa', '__label__Bluetooth_USB_Adapter', '__label__Board_mạch_điều_khiển', '__label__Body_liền_cho_bé', '__label__Boot_nữ', '__label__Bu_lông,_tắc_kê', '__label__Buồng_tắm_đa_năng', '__label__Buộc_tóc', '__label__Buộc_tóc_trẻ_em', '__label__Bài_vị_thờ', '__label__Bàn_Billiard_(Bi_A)', '__label__Bàn_cafe_(_café)', '__label__Bàn_chơi_game_-_Bàn_phòng_Game', '__

In [36]:
import os
MODEL_PATH = "models"

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

## 2. Naive Bayes

[https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html]


In [37]:
import pickle
import time
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1),
                                              max_df=0.8,
                                              max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                     ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Naive Bayes in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "naive_bayes.pkl"), 'wb'))

Done training Naive Bayes in 1.229372262954712 seconds.


## 3. Linear Classifier

[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html]


In [28]:
from sklearn.linear_model import LogisticRegression

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1),
                                              max_df=0.8,
                                              max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression(solver='lbfgs',
                                                multi_class='auto',
                                                max_iter=10000))
                     ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training Linear Classifier in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(
    MODEL_PATH, "linear_classifier.pkl"), 'wb'))

## 4. SVM
[https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html]

In [38]:
from sklearn.svm import SVC

start_time = time.time()
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 1),
                                              max_df=0.8,
                                              max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                     ])
text_clf = text_clf.fit(X_train, y_train)

train_time = time.time() - start_time
print('Done training SVM in', train_time, 'seconds.')

# Save model
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "svm.pkl"), 'wb'))

Done training SVM in 48.82425904273987 seconds.


## So sánh các mô hình

In [39]:
import numpy as np

# Naive Bayes
model = pickle.load(open(os.path.join(MODEL_PATH,"naive_bayes.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('Naive Bayes, Accuracy =', np.mean(y_pred == y_test))

# SVM
model = pickle.load(open(os.path.join(MODEL_PATH,"svm.pkl"), 'rb'))
y_pred = model.predict(X_test)
print('SVM, Accuracy =', np.mean(y_pred == y_test))

Naive Bayes, Accuracy = 0.22997329773030709
SVM, Accuracy = 0.503004005340454


In [40]:
# Xem kết quả trên từng nhãn
from sklearn.metrics import classification_report

nb_model = pickle.load(open(os.path.join(MODEL_PATH,"naive_bayes.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

ValueError: Number of classes, 1028, does not match size of target_names, 1971. Try specifying the labels parameter